In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Rheumatoid_Arthritis/GSE224842'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiles of peripheral blood mononuclear cells before abatacept treatment in rheumatoid arthritis patients."
!Series_summary	"To explore markers which predict the efficacy of abatacept in rheumatoid arthritis, peripheral blood mononuclear cells were obtained before abatacept treatment."
!Series_overall_design	"30 rheumatoid arthritis patients receiving abatacept were participated in the study. Blood samples were obtained before the initiation of abatacept treatment. Density-gradient separeted peripheral blood mononuclear cells were subjected the DNA microarray analyses."
Sample Characteristics Dictionary:
{0: ['disease state: rheumatoid arthritis'], 1: ['cell type: PBMC']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Define background from STEP 1 output
background = {
    '!Series_overall_design': "30 rheumatoid arthritis patients receiving abatacept were participated in the study. Blood samples were obtained before the initiation of abatacept treatment. Density-gradient separated peripheral blood mononuclear cells were subjected the DNA microarray analyses."
}

# Determine if gene expression data is available
if 'density-gradient separated peripheral blood mononuclear cells were subjected the DNA microarray analyses' in background['!Series_overall_design']:
    is_gene_available = True

# Check for availability of variables in the Sample Characteristics Dictionary
sample_characteristics = {0: ['disease state: rheumatoid arthritis'], 1: ['cell type: PBMC']}

# Rheumatoid Arthritis trait presence check:
if any('disease state' in value for value in sample_characteristics[0]):
    trait_row = 0

# The dataset does not provide age or gender information explicitly
age_row = gender_row = None

# Conversion functions for trait, age, and gender
def convert_trait(value):
    try:
        return 1 if 'rheumatoid arthritis' in value.lower() else 0
    except:
        return None

def convert_age(value):
    try:
        age_str = value.split(':')[-1].strip()
        return int(age_str)
    except:
        return None

def convert_gender(value):
    try:
        gender_str = value.split(':')[-1].strip().lower()
        if gender_str == 'female':
            return 0
        elif gender_str == 'male':
            return 1
        else:
            return None
    except:
        return None

# Save cohort information
save_cohort_info('GSE224842', './preprocessed/Rheumatoid_Arthritis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(
        clinical_data, 'Rheumatoid_Arthritis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Rheumatoid_Arthritis/trait_data/GSE224842.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM7034090': [1], 'GSM7034091': [1], 'GSM7034092': [1], 'GSM7034093': [1], 'GSM7034094': [1], 'GSM7034095': [1], 'GSM7034096': [1], 'GSM7034097': [1], 'GSM7034098': [1], 'GSM7034099': [1], 'GSM7034100': [1], 'GSM7034101': [1], 'GSM7034102': [1], 'GSM7034103': [1], 'GSM7034104': [1], 'GSM7034105': [1], 'GSM7034106': [1], 'GSM7034107': [1], 'GSM7034108': [1], 'GSM7034109': [1], 'GSM7034110': [1], 'GSM7034111': [1], 'GSM7034112': [1], 'GSM7034113': [1], 'GSM7034114': [1], 'GSM7034115': [1], 'GSM7034116': [1], 'GSM7034117': [1], 'GSM7034118': [1], 'GSM7034119': [1]}
